In [1]:
import pandas as pd

import regex as re
import wordcloud as wordcloud

from sklearn.model_selection import train_test_split


#import the data

#Step 7: Load the Data

df = pd.read_csv('train.csv')

#get the first 5 rows of the data
df.head(5)

#train_test_split(df, test_size=0.2, random_state = 42)


,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In the cell below, I am preprocessing the text data. Since twitter data has the @ mentions i will want to remove those. I will also want to remove the hashtags (but no need to remove the word after as that will preserve some of the data. I also want to remove unncessary whitespaces.

In [3]:


#remove the twitter handle from the text
#removes @
df['clean text'] = df['tweet']
#removes # without removing the word
#removes #
df['clean text'] = df['clean text'].apply(lambda x: re.sub(r'@[A-Za-z0-9]+','',x))
df['clean text'] = df['clean text'].apply(lambda x: re.sub("[^a-zA-Z]", " ", x))

#df['clean text'] = df['clean text'].apply(lambda x: re.sub(r'#\S+', '', x))
#df['clean text'] = df['clean text'].apply(lambda x: re.sub(r'http\S+', '', x))
#df['clean text'] = df['clean text'].apply(lambda x: re.sub(r'RT', '', x))
df['clean text'] = df['clean text'].apply(lambda x: re.sub(r'\n', '', x))


# tokenize the text
df['tokenized text'] = df['clean text'].apply(lambda x: x.split())

#join the tokens back together
df['clean text'] = df['tokenized text'].apply(lambda x: ' '.join(x))

#drop the tokenixed text column
df.drop(columns = ['tokenized text'], inplace = True)

#df.drop(columns = ['joined text'], inplace = True)
df




,id,label,tweet,clean text
0,1,0,@user when a father is dysfunctional and is s...,when a father is dysfunctional and is so selfi...
1,2,0,@user @user thanks for #lyft credit i can't us...,thanks for lyft credit i can t use cause they ...
2,3,0,bihday your majesty,bihday your majesty
3,4,0,#model i love u take with u all the time in ...,model i love u take with u all the time in ur
4,5,0,factsguide: society now #motivation,factsguide society now motivation
...,...,...,...,...
31957,31958,0,ate @user isz that youuu?ðððððð...,ate isz that youuu
31958,31959,0,to see nina turner on the airwaves trying to...,to see nina turner on the airwaves trying to w...
31959,31960,0,listening to sad songs on a monday morning otw...,listening to sad songs on a monday morning otw...
31960,31961,1,"@user #sikh #temple vandalised in in #calgary,...",sikh temple vandalised in in calgary wso conde...


In [4]:
import nltk
#remove the stop words
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem import PorterStemmer

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/cristinarivera/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:

stop_words=stopwords.words('english')
#add a stop word
stop_words.append('rt')
stop_words.append('amp')
stemmer=PorterStemmer()



In [6]:
#remove the stop words
df['clean text'] = df['clean text'].apply(lambda x: ' '.join([stemmer.stem(i) for i in x.split() if i not in stop_words]))

df

,id,label,tweet,clean text
0,1,0,@user when a father is dysfunctional and is s...,father dysfunct selfish drag kid dysfunct run
1,2,0,@user @user thanks for #lyft credit i can't us...,thank lyft credit use caus offer wheelchair va...
2,3,0,bihday your majesty,bihday majesti
3,4,0,#model i love u take with u all the time in ...,model love u take u time ur
4,5,0,factsguide: society now #motivation,factsguid societi motiv
...,...,...,...,...
31957,31958,0,ate @user isz that youuu?ðððððð...,ate isz youuu
31958,31959,0,to see nina turner on the airwaves trying to...,see nina turner airwav tri wrap mantl genuin h...
31959,31960,0,listening to sad songs on a monday morning otw...,listen sad song monday morn otw work sad
31960,31961,1,"@user #sikh #temple vandalised in in #calgary,...",sikh templ vandalis calgari wso condemn act


In [32]:
#get the hate speech labels only and put them in a list
#hate speech is 1

from sklearn.feature_extraction.text import CountVectorizer

#frequency representation
from nltk.stem import WordNetLemmatizer
cv=CountVectorizer(max_features=1500)
#get the bag of words
X=cv.fit_transform(df['clean text']).toarray()

#sort the words in the dictionary from most frequent to least frequent
word_freq={k: v for k, v in sorted(cv.vocabulary_.items(),reverse=True, key=lambda item: item[1])}



#X.shape




In [33]:
from wordcloud import WordCloud
hate_speech_df = df[df['label'] == 1]
hate_speech_df.drop(columns = ['tweet'], inplace = True)
not_hate_speech_df = df[df['label'] == 0]
print(hate_speech_df)
#get the hate speech frequencies
hate_speech_freq = hate_speech_df['clean text'].tolist()


#get the hate speech word frequencies and put them in a dictionary
hate_speech_word_freq = {}
for i in hate_speech_freq:
    for j in i.split():
        if j in hate_speech_word_freq:
            hate_speech_word_freq[j] += 1
        else:
            hate_speech_word_freq[j] = 1
#print(hate_speech_word_freq)


#print word frequencies
hate_speech_word_freq1 = {k: v for k, v in sorted(hate_speech_word_freq.items(),reverse=True, key=lambda item: item[1])}

print(hate_speech_word_freq1)
wc = WordCloud(width=800, height=400, max_words=200,stopwords=['like']).generate_from_frequencies(hate_speech_word_freq1)
img = wc.to_image()
img.show()
#save the image
img.save("hate_speech_wordcloud.png")

/var/folders/8x/smqvb831281gbpj31c5yz_600000gn/T/ipykernel_45901/3318217625.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hate_speech_df.drop(columns = ['tweet'], inplace = True)


          id  label                                         clean text
13        14      1  cnn call michigan middl school build wall chan...
14        15      1  comment australia opkillingbay seashepherd hel...
17        18      1                                       retweet agre
23        24      1                              lumpi say prove lumpi
34        35      1  unbeliev st centuri need someth like neverump ...
...      ...    ...                                                ...
31934  31935      1            ladi ban kentucki mall jcpenni kentucki
31946  31947      1   omfg offend mailbox proud mailboxprid liberalism
31947  31948      1    ball hashtag say weasel away lumpi toni dipshit
31948  31949      1                  make ask anybodi god oh thank god
31960  31961      1        sikh templ vandalis calgari wso condemn act

[2242 rows x 3 columns]
{'trump': 217, 'white': 159, 'libtard': 150, 'black': 149, 'like': 144, 'racist': 114, 'polit': 101, 'peopl': 95, 'liber': 

In [35]:
from sklearn.model_selection import train_test_split
#Step 9: Extract the features and target
y = df['label'].values
#X was defined previosly as the bag of words
#Step 10: Split the data into training set and test set.
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)


In [36]:
from sklearn.naive_bayes import MultinomialNB
model=MultinomialNB()
model.fit(X_train,y_train)
y_pred_naivebayes=model.predict(X_test)

In [56]:
from sklearn.metrics import accuracy_score
naive_bayes_ac = accuracy_score(y_test,y_pred_naivebayes)
print('naive bayes accuracy score:',naive_bayes_ac)

naive bayes accuracy score: 0.9368058814328172


In [41]:

from sklearn.linear_model import LogisticRegression

simplelogistic =LogisticRegression();
simplelogistic.fit(X_train,y_train)

y_pred_log=simplelogistic.predict(X_test)


In [42]:

lr_accuracy_score = accuracy_score(y_test,y_pred_log)
print(lr_accuracy_score)

0.9488503050211169


In [43]:
from sklearn.neural_network import MLPClassifier

#Step 13: Create a slightly complicated model and check its accuracy
simpleneuralnetwork = MLPClassifier(hidden_layer_sizes=(5));
simpleneuralnetwork.fit(X_train,y_train)
y_pred_NN=simpleneuralnetwork.predict(X_test)


/Users/cristinarivera/.julia/conda/3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [44]:
nn_accuracy_score = accuracy_score(y_test,y_pred_NN)
print("neural network accuracy score:",nn_accuracy_score)

neural network accuracy score: 0.9371187236039418


In [45]:

from sklearn.model_selection import GridSearchCV

NB=MultinomialNB()

params1 = {'alpha':[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]}
grid_search_NB = GridSearchCV(estimator=NB,param_grid=params1,scoring='accuracy',verbose=10)
grid_search_NB.fit(X_train,y_train)



Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5; 1/10] START alpha=0.1..................................................
[CV 1/5; 1/10] END ...................alpha=0.1;, score=0.938 total time=   0.7s
[CV 2/5; 1/10] START alpha=0.1..................................................
[CV 2/5; 1/10] END ...................alpha=0.1;, score=0.942 total time=   0.3s
[CV 3/5; 1/10] START alpha=0.1..................................................
[CV 3/5; 1/10] END ...................alpha=0.1;, score=0.945 total time=   0.3s
[CV 4/5; 1/10] START alpha=0.1..................................................
[CV 4/5; 1/10] END ...................alpha=0.1;, score=0.938 total time=   0.3s
[CV 5/5; 1/10] START alpha=0.1..................................................
[CV 5/5; 1/10] END ...................alpha=0.1;, score=0.941 total time=   0.3s
[CV 1/5; 2/10] START alpha=0.2..................................................
[CV 1/5; 2/10] END ...................alpha=0.2;

GridSearchCV(estimator=MultinomialNB(),
             param_grid={'alpha': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9,
                                   1.0]},
             scoring='accuracy', verbose=10)

In [53]:
# grid search results
grid_result = pd.DataFrame(grid_search_NB.cv_results_)
grid_result[['params','mean_test_score']]

,params,mean_test_score
0,{'alpha': 0.1},0.940788
1,{'alpha': 0.2},0.940827
2,{'alpha': 0.3},0.940084
3,{'alpha': 0.4},0.939614
4,{'alpha': 0.5},0.939028
5,{'alpha': 0.6},0.938519
6,{'alpha': 0.7},0.938754
7,{'alpha': 0.8},0.938363
8,{'alpha': 0.9},0.938206
9,{'alpha': 1.0},0.937933


In [55]:
#get the best parameters
best_params = grid_search_NB.best_params_
print("best perimeters",best_params)

#gridsearch_logistic_results = gridsearch_logistic.grid_scores_

best perimeters {'alpha': 0.2}


In [29]:

from sklearn.metrics import classification_report
cf=classification_report(y_test,y_pred_naivebayes)
print(cf)

              precision    recall  f1-score   support

           0       0.97      0.97      0.97      5937
           1       0.63      0.58      0.60       456

    accuracy                           0.95      6393
   macro avg       0.80      0.78      0.79      6393
weighted avg       0.94      0.95      0.94      6393



In [30]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,y_pred_naivebayes)
print(cm)

[[5783  154]
 [ 193  263]]


In [31]:
from sklearn.metrics import accuracy_score
nb_acc = accuracy_score(y_test,y_pred_naivebayes)

0.9457218833098702

In [ ]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

logreg = LogisticRegression()

params = {'C': [0.01, 0.1, 0.5, 0.9],'solver':["newton-cg", "lbfgs", "liblinear"]}

#solver = ['newton-cg', 'lbfgs', 'liblinear']
print(X_train.shape)
print(y_train.shape)

gridsearch_logistic = GridSearchCV(logreg,param_grid=params,verbose=10,scoring='accuracy',)
gridsearch_logistic.fit(X_train,y_train)